In [14]:
from openai import OpenAI
import openai
import base64
import os
import requests
import pandas as pd
import io
import glob
import time
import re
from pathlib import Path
from pdf2image import convert_from_path

with open("../../api_key.txt", "r") as file:
    api_key = file.read().strip()

# Set the API key as an environment variable
os.environ['OPENAI_API_KEY'] = api_key
client = OpenAI()

In [20]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "../../data/primary_sources/Paris_1878_Italy_7.jpg"

In [21]:
# Convert to JPG from source
images = convert_from_path('../../data/primary_sources/istat94_tav3.pdf')

for count, image in enumerate(images):
    image.save(f'../../data/primary_sources/istat94_tav3_{count}.jpg', 'JPEG')

In [44]:
# try this approach then..
def chatgpt_csv(file_path):
    base64_image = encode_image(file_path)
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Parse this image to .csv with \t as a delimiter. Put the Years in columns. Create a separate column for Province Name. Delimit the csv file by ```csv at the beginning and ``` at the end."
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                        "detail":"high"
                    }
                }
            ]
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=messages,
        max_tokens=4096
    )
    csv_content = response.choices[0].message.content
    csv_string = csv_content.split('```csv\n')[1].split('```')[0]
    df = pd.read_csv(io.StringIO(csv_string), delimiter='\t')
    df['Pdf'] = os.path.basename(file_path)
    output_path = os.path.join("../../data/gpt_ocr/", os.path.basename(file_path) + ".csv")
    df.to_csv(output_path, sep="\t", index=False)
    return df

In [45]:
directory = '../../data/primary_sources/'
pattern = '*.jpg'  # Pattern to match

# Use glob to list all files matching the pattern
matching_files = glob.glob(os.path.join(directory, pattern))
pattern = re.compile(r'istat94_tav3_(\d+)\.jpg')

# Filter files where the number is 5 or greater
filtered_files = [file for file in matching_files if pattern.search(file)]
filtered_files.sort()

filtered_files[0]

'../../data/primary_sources/istat94_tav3_0.jpg'

In [46]:
chatgpt_csv(filtered_files[0])

,Province Name,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,Pdf
0,Agliè,3545,4088,3647,3428,3152,276,2594,2463,2805,2578,2623,istat94_tav3_0.jpg
1,Airasca,1547,1565,1749,1851,1831,1839,1850,1689,1645,1501,1532,istat94_tav3_0.jpg
2,Ala di Stura,1031,1018,1248,1205,1143,753,1047,1053,1042,974,1010,istat94_tav3_0.jpg
3,Albiano d'Ivrea,2153,2515,2278,2400,2249,1837,1484,1400,1381,1580,1504,istat94_tav3_0.jpg
4,Alice Superiore,1471,1423,1404,1194,1385,1871,880,729,678,588,641,istat94_tav3_0.jpg
5,Almese,2790,3482,2048,3046,3083,2807,2586,2589,2417,3200,3332,istat94_tav3_0.jpg
6,Alpette,592,603,619,601,700,828,688,423,579,641,549,istat94_tav3_0.jpg
7,Andezeno,1826,1388,1298,2188,2844,2321,3109,2212,2728,17260,14722,istat94_tav3_0.jpg
8,Arignano,1687,1800,1710,1305,1341,1219,1113,927,812,1337,1699,istat94_tav3_0.jpg
9,Avigliana,7958,6792,7053,7240,2318,1045,1941,781,978,585,883,istat94_tav3_0.jpg


In [12]:
for file in filtered_files[1:]:
    try:
        result = chatgpt_csv(file)
        print(f"Processed file: {file}")
    except Exception as e:
        print(f"Failed to process file: {file} with error: {e}")


Failed to process file: . with error: name 'encode_image' is not defined
Failed to process file: . with error: name 'encode_image' is not defined
Failed to process file: / with error: name 'encode_image' is not defined
Failed to process file: . with error: name 'encode_image' is not defined
Failed to process file: . with error: name 'encode_image' is not defined
Failed to process file: / with error: name 'encode_image' is not defined
Failed to process file: d with error: name 'encode_image' is not defined
Failed to process file: a with error: name 'encode_image' is not defined
Failed to process file: t with error: name 'encode_image' is not defined
Failed to process file: a with error: name 'encode_image' is not defined
Failed to process file: / with error: name 'encode_image' is not defined
Failed to process file: p with error: name 'encode_image' is not defined
Failed to process file: r with error: name 'encode_image' is not defined
Failed to process file: i with error: name 'encode_